In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean

In [2]:
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
nca = NeighborhoodComponentsAnalysis(random_state=42)
X_train_nca = nca.fit_transform(X_train, y_train)

In [5]:
def classify_test_object(test_object):
    class_probabilities = {}
    for class_label in np.unique(y_train):
        class_indices = np.where(y_train == class_label)[0]
        distances = [euclidean(test_object, X_train_nca[i]) for i in class_indices]        
        prob = 1 / (np.mean(distances) + 1e-6)
        class_probabilities[class_label] = prob
    return max(class_probabilities, key=class_probabilities.get)


In [6]:
y_pred = [classify_test_object(test_obj) for test_obj in X_test]
accuracy = np.mean(y_pred == y_test)
print(f"Acurácia do classificador: {accuracy:.2f}")

Acurácia do classificador: 0.29


In [7]:
from classNCA import NCA

In [8]:
y_train[:12]

array([1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0])

In [9]:
NCA().fit(X_train[:12], y_train[:12])

[[ True False False  True False  True False  True False False  True False]
 [False  True  True False  True False  True False False  True False False]
 [False  True  True False  True False  True False False  True False False]
 [ True False False  True False  True False  True False False  True False]
 [False  True  True False  True False  True False False  True False False]
 [ True False False  True False  True False  True False False  True False]
 [False  True  True False  True False  True False False  True False False]
 [ True False False  True False  True False  True False False  True False]
 [False False False False False False False False  True False False  True]
 [False  True  True False  True False  True False False  True False False]
 [ True False False  True False  True False  True False False  True False]
 [False False False False False False False False  True False False  True]]


1